In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mlops/.ivy2/cache
The jars for the packages stored in: /home/mlops/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f9e64cea-5a8e-4256-8ccd-0c1eb7195e46;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 237ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      def

In [4]:
ticketmaster = spark.read.format('delta').load('/home/mlops/project/DeltaLake/gold_data/ticketmaster_flattened_table')
ticketmaster.to_pandas_on_spark()

22/02/02 19:19:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/02 19:19:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,spotify_followers,spotify_genres,spotify_href,spotify_id,spotify_name,spotify_popularity,spotify_type,spotify_uri,ticketmaster_name,ticketmaster_id,ticketmaster_locale,ticketmaster_distance,ticketmaster_units,ticketmaster_price_min,ticketmaster_price_max,spotify_url,spotify_image_url,ticketmaster_date,ticketmaster_venue_name,ticketmaster_venue_id,ticketmaster_venue_url,ticketmaster_venue_postal_code,ticketmaster_venue_timezone,ticketmaster_venue_city,ticketmaster_venue_country,ticketmaster_venue_address,ticketmaster_url,ticketmaster_artist_locale,ticketmaster_venue_latitude,ticketmaster_venue_longitude
0,NaN,None,None,None,None,NaN,None,None,SEB MELLIA,rZ7SnyZ1AduEvs,fr-fr,66.01,MILES,25.00,39.00,None,None,2023-02-04,PALAIS DES CONGRES-SALLE ERASME,rZFSnyZeek6,https://www.ticketmaster.fr/fr/salle/palais-de...,67000,Europe/Paris,Strasbourg,France,PLACE DE BORDEAUX - WACKEN,None,None,48.598322,7.760900
1,68207.0,"[album rock, mellow gold, new wave pop, philly...",https://api.spotify.com/v1/artists/7uhvDINTTiD...,7uhvDINTTiD0XBrP9fquN1,The Hooters,56.0,artist,spotify:artist:7uhvDINTTiD0XBrP9fquN1,The Hooters,Z698xZC2Z17uPao,en-us,67.49,MILES,35.00,35.00,https://open.spotify.com/artist/7uhvDINTTiD0XB...,https://i.scdn.co/image/ab6761610000e5eb15896b...,2022-06-28,Musiktheater REX,Z698xZC2Za7r9,https://www.ticketmaster.de/venue/musiktheater...,64625,Europe/Berlin,Bensheim,Germany,Fabrikstraße 10,http://www.ticketmaster.de/artist/918988,en-us,49.680740,8.619640
2,794733.0,"[francoton, pop urbaine]",https://api.spotify.com/v1/artists/7gU9VyFRN3J...,7gU9VyFRN3JWPJ5oHOil60,Tayc,75.0,artist,spotify:artist:7gU9VyFRN3JWPJ5oHOil60,TAYC,rZ7SnyZ1Ad700d,fr-fr,67.72,MILES,26.00,26.00,https://open.spotify.com/artist/7gU9VyFRN3JWPJ...,https://i.scdn.co/image/ab6761610000e5ebce9a6f...,2021-12-16,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",http://www.ticketmaster.fr/artist/1044372,en-us,48.576090,7.729777
3,883293.0,"[electro swing, nu jazz]",https://api.spotify.com/v1/artists/37J1PlAkhRK...,37J1PlAkhRK7yrZUtqaUpQ,Caravan Palace,66.0,artist,spotify:artist:37J1PlAkhRK7yrZUtqaUpQ,CARAVAN PALACE,rZ7SnyZ1Adk7qv,fr-fr,67.72,MILES,34.20,34.20,https://open.spotify.com/artist/37J1PlAkhRK7yr...,https://i.scdn.co/image/ab6761610000e5ebdedbe7...,2021-12-18,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",http://www.ticketmaster.fr/artist/141183,en-us,48.576090,7.729777
4,145760.0,"[cali rap, southern hip hop]",https://api.spotify.com/v1/artists/1grN0519h2z...,1grN0519h2zYqpRtYbDZAl,Larry June,68.0,artist,spotify:artist:1grN0519h2zYqpRtYbDZAl,LARRY,rZ7SnyZ1AduOGE,fr-fr,67.72,MILES,26.00,26.00,https://open.spotify.com/artist/1grN0519h2zYqp...,https://i.scdn.co/image/ab6761610000e5eb1c0b98...,2022-01-15,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",None,None,48.576090,7.729777
5,43856.0,[sigilkore],https://api.spotify.com/v1/artists/4qUoZ6ErhTY...,4qUoZ6ErhTYFhAI0W1iI9q,Ezekiel,67.0,artist,spotify:artist:4qUoZ6ErhTYFhAI0W1iI9q,EZ3KIEL - EZ3KIEL 'LA MEMOIRE DU FEU',rZ7SnyZ1AdG44x,fr-fr,67.72,MILES,27.20,27.20,https://open.spotify.com/artist/4qUoZ6ErhTYFhA...,https://i.scdn.co/image/ab6761610000e5ebed06c7...,2022-01-20,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",http://www.ticketmaster.fr/artist/946675,en-us,48.576090,7.729777
6,40801.0,"[electro swing, nu jazz]",https://api.spotify.com/v1/artists/6jX8VHUJomY...,6jX8VHUJomYSfi5Hobdmmn,Lyre Le Temps,47.0,artist,spotify:artist:6jX8VHUJomYSfi5Hobdmmn,LYRE LE TEMPS,rZ7SnyZ1AdGyqo,fr-fr,67.72,MILES,17.00,17.00,https://open.spotify.com/artist/6jX8VHUJomYSfi...,https://i.scdn.co/image/ab6761610000e5eb78c466...,2022-01-21,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-

In [5]:
ticketmaster_cleansed = ticketmaster

ticketmaster_cleansed = ticketmaster_cleansed.na.drop(subset=["spotify_id", "spotify_image_url"])

ticketmaster_cleansed.count()

683

In [6]:
import pyspark.sql.functions as F

ticketmaster_cleansed = ticketmaster_cleansed.filter(F.size("spotify_genres") > 0)

ticketmaster_cleansed.count()

552

In [7]:
ticketmaster_cleansed.write.format("delta").mode("overwrite").save("/home/mlops/project/DeltaLake/gold_data/ticketmaster_cleansed_table")

In [6]:

ticketmaster = spark.read.format('delta').load('/home/mlops/project/DeltaLake/gold_data/ticketmaster_cleansed_table')
ticketmaster.show()


+-----------------+--------------------+--------------------+--------------------+----------------+------------------+------------+--------------------+--------------------+---------------+-------------------+---------------------+------------------+----------------------+----------------------+--------------------+--------------------+-----------------+-----------------------+---------------------+----------------------+------------------------------+---------------------------+-----------------------+--------------------------+--------------------------+--------------------+--------------------------+---------------------------+----------------------------+
|spotify_followers|      spotify_genres|        spotify_href|          spotify_id|    spotify_name|spotify_popularity|spotify_type|         spotify_uri|   ticketmaster_name|ticketmaster_id|ticketmaster_locale|ticketmaster_distance|ticketmaster_units|ticketmaster_price_min|ticketmaster_price_max|         spotify_url|   spotify_imag